In [50]:
import pandas as pd
import numpy as np
import plotly.express as px
import dash 
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc
from dash import dash_table

In [11]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [45]:
df = pd.read_csv('priceFIPS.csv')
df['FIPS'] = df['FIPS'].apply(lambda x: str(x).zfill(5))

In [53]:
total = np.insert(df.state.unique(),0,'USA')

In [55]:
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dcc.Dropdown(id='state_dropdown',
                 options=[{'label': state, 'value': state}
                          for state in total]),
    dcc.Graph(id='graph'),
])

@app.callback(
    Output('graph', 'figure'),
    Input('state_dropdown', 'value'))
def update_figure(state):
    if state is None or state =='USA':
        fig = px.choropleth(df, geojson=counties, locations='FIPS', color='price',
                                   color_continuous_scale="jet",
                            hover_name="county",
                                   range_color=(df['price'].min()-0.1, df['price'].max()+0.1),
                                   scope="usa",
                            title = 'USA gas price by county'
                                  )
        fig.update_traces(marker_line_width=0.1, marker_opacity=0.8)
        fig.update_geos(showsubunits=True, subunitcolor="black")
        fig.update_layout(height=600, margin={"r":0,"t":50,"l":0,"b":0})
    else:
        df2 = df[df['state']==state]
        fig = px.choropleth(df2, geojson=counties, locations='FIPS', color='price',
                                   color_continuous_scale="jet",
                            hover_name="county",
                                   range_color=(df['price'].min()-0.1, df['price'].max()+0.1),
                            title ='gas price by county in State ' + state
                                  )
        fig.update_geos(fitbounds='locations', visible=False)
        fig.update_traces(marker_line_width=0.3)
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', height= 500, width = '80%')